In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data_Washington Fatal Crash Survey.csv", low_memory=False)
zips = pd.read_csv("US%20Zip%20Codes%20from%202013%20Government%20Data.txt")

In [9]:
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS



#'methdrug', 'drugsts', drugtst1-12, drugres1-12, 'alcres', 'alcsts', 'alctst'
#ejectpath alcmeth
#investjur repjur

We will drop all columns that are more than 90% NANs because there are not enough values for them to be significant. We'll also drop columns that are not relevant to the problem statement: ['hosp_tm','arr_tm','not_tm','investjur','repjur','lab','methdrug','alcmeth','drugsts','drugtst1']

## Derive zip code for crashes 

Method: Reverse geocoding with ArcGIS and impute any missing zipcode values

In [10]:
# long : x
# lat : y

# driver zip code : dzip 
df.rename(columns={'x': 'lon', 'y': 'lat'}, inplace=True)

In [12]:
gis = GIS(api_key="AAPK206d8c337d5a416ca8a7824330c0ca7bkQWmIxSD8K5_qxX1ly5btaWLcl36cVX_1iDwCQxl_IgY56Zh6TN-Usx26i4oM77l")
# exposed for now

def get_zip(df,lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]),"y":float(df[lat_field]), "spatialReference":{"wkid":4326}})))
    return location['address']['Postal']

df['crashzip'] = df.apply(get_zip, axis=1, lat_field='lat', lon_field='lon')

In [83]:
# update missing crash zip values in df 
# case 78 03/18/2020 in county 45(Pacific) --> crash lat long improperly recorded  78.304936,  -778.304936 impossible! city = unincorporated
# FOREST SERVICES RD 2300 is not an actual road... should we drop the row?
# case 349 09/12/2021 in county 5 --> zip: 99352 could not be identified by arcGIS identified using haversine distance
# case 638 09/05/2021 in county 19(Ferry) --> lat long recorded as 78.304936,  -778.304936 impossible! city = unincorporated
# Ferry county is on a reservation in the middle of nowhere next to a natl forest far northeast of WA 
# crash occurred on BRIDGE CREEK RD in Ferry County --> brute force zip: 99138

df.loc[3690,"crashzip"] = 99352
df.loc[4128,"crashzip"] = 99138

In [84]:
# saved because computationally expensive 
# don't want to call API every time we run the code 
# DONT RUN THAT PART 

df.to_csv('crashdata_updated.csv') # import THIS data set

## What proportion of drivers involved in fatal crashes crash in communities where they live?

Method: Calculate proportion mathematically

In [85]:
# start from here

data = pd.read_csv("crashdata_updated.csv", low_memory=False) # some cols have mixed dtypes...
data["crash_dt"]= pd.to_datetime(data["crash_dt"])

In [98]:
# Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?

# How do we want to define "community"? Zip code should be fine... 

data['dzip'] = data['dzip'].astype(str)
# get rid of the trailing zeros
data['dzip'] = data['dzip'].replace(r'\.0$', '', regex=True)

In [110]:
print("The proportion of drivers involved in fatal crashes where they don't live = {}".format(len(data.query('dzip != crashzip'))/len(data)))

The proportion of drivers involved in fatal crashes where they don't live = 0.7628267182962246


In [109]:
print("The proportion of drivers involved in fatal crashes where they live = {}".format(len(data.query('dzip == crashzip'))/len(data)))

The proportion of drivers involved in fatal crashes where they live = 0.2371732817037754


- Calculate the same proportions for before Spring 2020 and after Spring 2020
- Represent these 3 proportions in a horizontal stacked bar chart 

In [ ]:
df = df.loc[:, df.isnull().mean() < .9]

## Is there a difference between types of crashes and behavior factors among residents vs visitors?

Method: Chi-Squared Test for Homogeneity between crashes among residents and visitors

- null hypothesis: Visitor and Resident populations are homogeneous regarding the proportions of categories of categorical variables (crash/behavior factors)

In [146]:
# Create a categorical variable to represent whether driver is local resident or visitor
# "resident" = local resident
# "visitor" = out of town

drive_res = [] # set empty list 
for index, row in data.iterrows():
    if row['dzip']==row['crashzip']:
        drive_res.append("resident") # if equal add "resident" to list
    else:
        drive_res.append("visitor") # else add "visitor"
        
data["d_res"] = drive_res # cast list as df column

In [203]:
# method: chi-squared test of homogeneity 

# Is there a difference between the types of crashes among residents vs visitors?
# Is there a difference between the behavior factors among residents vs visitors?

In [215]:
# experimental crosstabs to look at the data lol

# crash level
crash_tab = pd.crosstab([data.numfatal,data.roadclass,data.pv_inv,data.ped_inv,data.ht_inv,data.bike_inv,data.mc_inv],data.d_res, rownames = ["Fatalities", "RoadClass","PV_inv","PED_inv","HT_inv","Bike_inv", "MC_inv"],colnames = ["Driver"],margins=True, margins_name="Total")

crashtype = pd.crosstab([data.numfatal,data.crashtype],data.d_res, rownames = ["Fatalities", "Crashtype"],colnames = ["Driver"],margins=True, margins_name="Total")

# driver behavior level
behav_tab = pd.crosstab([data.dd_inv,data.speed_inv,data.drowsy_inv,data.drdist_inv],data.d_res, rownames = ["Drinking","Speeding","Drowsy","Distract"],colnames = ["Driver"],margins=True, margins_name="Total")

In [216]:
crashtype

Driver                resident  visitor  Total
Fatalities Crashtype                          
1          0                 4       15     19
           1                89      235    324
           2                34       93    127
           3                 1        1      2
           5                 1        1      2
...                        ...      ...    ...
4          50                0        1      1
           51                1        0      1
           86                0        1      1
           87                0        1      1
Total                      980     3152   4132

[115 rows x 3 columns]

In [206]:
behav_tab

Driver                             resident  visitor  Total
Drinking Speeding Drowsy Distract                          
0        0        0      0              355     1179   1534
                         1              149      432    581
                  1      0               15       35     50
                         1                0        3      3
         1        0      0              124      497    621
                         1               15       64     79
                  1      0                2        4      6
1        0        0      0              126      456    582
                         1               46      101    147
                  1      0                2        6      8
         1        0      0              137      332    469
                         1                9       43     52
Total                                   980     3152   4132

## Are there specific resident ZIP Codes that tend to produce higher-risk drivers that are involved in fatal crashes at a higher rate?

- Keywords: produce... tend... and rate...

- For simplicity we'll just assume that "high risk" = all of these drivers since they all killed at least 1 person 
- Calculate: # of high risk drivers/

In [213]:
data['crashtype'].nunique()

58